In [20]:
import pandas as pd
import numpy as np
table_all_current = pd.read_pickle('table_all_current.pkl')

In [50]:
def expected_portfolio_return_evenly_weighted(table_all_current, avail_funds):
    evenly_weighted_expected_returns = []
    for idx in table_all_current.index:
        evenly_weighted_expected_returns.append(
            (table_all_current['return_preds'][idx]) * (avail_funds/len(table_all_current)))
    expected_portfolio_return = sum(evenly_weighted_expected_returns)/avail_funds
    return expected_portfolio_return

def rank_table_by_shrop_ratio_RAR(table_all_current, avail_funds):
    table_all_current['shrop_ratio'] = np.nan
    for idx in table_all_current.index:
        table_all_current['shrop_ratio'][idx] = (
            (expected_portfolio_return_evenly_weighted(table_all_current, avail_funds) - 0.0188) / 
                                                        (table_all_current['prob_default'][idx]))
    table_all_current_ranked = table_all_current.sort_values(by='shrop_ratio',axis=0, ascending=False)
    return table_all_current_ranked

def recommended_loans_ranked_by_shrop_RAR(table_all_current, max_prob_default, min_desired_return, avail_funds):
    rec_table = table_all_current[(table_all_current['prob_default'] <= max_prob_default) & 
                             (table_all_current['return_preds'] >= min_desired_return)]
    rec_table_ranked = rank_table_by_shrop_ratio_RAR(rec_table, avail_funds)
    return rec_table_ranked

def portfolio_prob_default_evenly_weighted(table_all_current, avail_funds):
    evenly_weighted_prob_default = []
    for idx in table_all_current.index:
        evenly_weighted_prob_default.append(
            (table_all_current['prob_default'][idx]) * (avail_funds/len(table_all_current)))
    portfolio_prob_default = sum(evenly_weighted_prob_default)/avail_funds
    return portfolio_prob_default

def portfolio_shrop_ratio_evenly_weighted(table_all_current, avail_funds):
    table_all_current_ranked = rank_table_by_shrop_ratio_RAR(table_all_current, avail_funds)
    evenly_weighted_shrop_ratio = []
    for idx in table_all_current_ranked.index:
        evenly_weighted_shrop_ratio.append(
            (table_all_current['shrop_ratio'][idx]) * (avail_funds/len(table_all_current)))
    portfolio_shrop_ratio = sum(evenly_weighted_shrop_ratio)/avail_funds
    return portfolio_shrop_ratio

def summarize_recommendation(table_all_current, max_prob_default, min_desired_return, avail_funds):
    rec_table_ranked = recommended_loans_ranked_by_shrop_RAR(table_all_current, max_prob_default, min_desired_return, avail_funds)
    port_prob_def = portfolio_prob_default_evenly_weighted(rec_table_ranked,avail_funds)
    port_exp_return = expected_portfolio_return_evenly_weighted(rec_table_ranked, avail_funds)
    port_shrop_ratio = portfolio_shrop_ratio_evenly_weighted(rec_table_ranked,avail_funds)
    max_investable = rec_table_ranked['loan_amnt'].sum() - rec_table_ranked['funded_amnt'].sum()
    return (rec_table_ranked,port_prob_def,port_exp_return,port_shrop_ratio,max_investable)

In [51]:
(rec_table_ranked,port_prob_def,
 port_exp_return,port_shrop_ratio,max_investable) = summarize_recommendation(table_all_current,
                                                                             max_prob_default=0.2,
                                                                             min_desired_return=0.07,
                                                                             avail_funds=10000)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_

In [52]:
rec_table_ranked

,prob_default,prob_fullypaid,return_preds,loan_amnt,funded_amnt,int_rate,emp_length,annual_inc,dti,delinq_2yrs,...,emp_title_2_Truck Driver,emp_title_2_Vice President,emp_title_2_driver,emp_title_2_manager,emp_title_2_owner,emp_title_2_sales,emp_title_2_supervisor,emp_title_2_teacher,emp_title_2_truck driver,shrop_ratio
id,,,,,,,,,,,,,,,,,,,,,
153385542,0.006103,0.993897,0.105928,20000,18350.0,0.1524,4.0,52000.0,32.24,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.596288
154586252,0.006614,0.993386,0.086362,23000,19800.0,0.1308,5.0,65000.0,5.58,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.469490
154750280,0.010279,0.989721,0.105304,40000,33950.0,0.1524,1.0,120000.0,20.76,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.666869
154759877,0.013345,0.986655,0.080908,22100,8050.0,0.1695,6.0,125000.0,28.25,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.675606
154882668,0.019517,0.980483,0.078335,13275,6625.0,0.1308,8.0,50000.0,6.60,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.564561
154607967,0.019852,0.980148,0.145405,30000,16675.0,0.2000,4.0,160000.0,26.64,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.487403
154334942,0.045072,0.954928,0.146274,30000,20925.0,0.2000,2.0,100000.0,26.41,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.976516
154876971,0.108523,0.891477,0.115091,7000,5125.0,0.1774,1.0,66000.0,8.36,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.820885
154731223,0.109099,0.890901,0.092688,25525,8475.0,0.1430,1.0,35000.0,33.30,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.816557


In [53]:
port_prob_def

0.05815220962871206

In [54]:
port_exp_return

0.1078851933745725

In [55]:
port_shrop_ratio

5.207889933934658

In [56]:
max_investable

95000.0